<a href="https://colab.research.google.com/github/Andrvz/dsVaz/blob/main/ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Exploratórias e Visualização de Dados para E-Commerce




# Manipulando Banco de Dados

> ### **Bibliotecas** 
-   **Pandas**: 
-   **DuckDB**: 
-   **Matplotlib**: 
-   **Seaborn**: 

In [1]:
# !pip install duckdb

In [2]:
%%time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
import duckdb
import warnings

CPU times: user 752 ms, sys: 204 ms, total: 956 ms
Wall time: 849 ms


In [3]:
%%time
# Carregar dataset
cliente_df = pd.read_csv(r'sample_data/cliente.csv', encoding='ISO-8859-1', sep=';', header=0)
loja_df = pd.read_csv(r'sample_data/loja.csv', encoding='ISO-8859-1', sep=';')
produto_df = pd.read_csv(r'sample_data/produto_1.csv', encoding='ISO-8859-1', sep=';')
tempo_df = pd.read_csv(r'sample_data/dm_tempo_2.csv', encoding='ISO-8859-1', sep=';')
vendas_df = pd.read_csv(r'sample_data/vendas.csv', encoding='ISO-8859-1', sep=';')

# Tratar dataset
vendas_df['Receita_venda'] = vendas_df['Receita_venda'].apply(lambda x : x[1:])
tempo_df['Data'] = tempo_df['Data'].apply(lambda x : datetime.strptime(x, "%d/%m/%Y"))

# Definir tipos de dados para colunas
cliente_df = cliente_df.astype({'ID_Cliente ': 'int64', 'Estado Civil': 'object', 'Sexo': 'object', 'Bairro': 'object'})
loja_df = loja_df.astype({'ID_Loja': 'int64', 'COD_Loja': 'int64', 'NM_Loja': 'object'})
produto_df = produto_df.astype({'ID_Produto': 'int64', 'COD_Produto': 'int64', 'NM_Produto': 'string', 'Secao': 'string', 'Grupo': 'string', 'Subgrupo': 'string'})
tempo_df = tempo_df.astype({'ID_Tempo': 'int64', 'Ano': 'int64', 'Mês': 'int32', 'Dia': 'int32', 'Data': 'string'})
vendas_df = vendas_df.astype({'ID_Loja': 'int64', 'ID_Produto': 'int64', 'ID_Cliente': 'int64', 'ID_Tempo': 'int64', 'Qtde_Vendida': 'string', 'Receita_venda': 'float'})

# Renomear colunas do dataset
cliente_df.rename(columns = {'ID_Cliente ':'ID_Cliente', 'Estado Civil':'Estado_Civil'}, inplace = True)
tempo_df.rename(columns = {'Mês':'Mes', 'Data':'Data_sql'}, inplace = True)

CPU times: user 19.7 ms, sys: 0 ns, total: 19.7 ms
Wall time: 19.7 ms


In [4]:
%%time
duckdb.sql("""
-- DROP TABLE cliente;
-- DROP TABLE loja;
-- DROP TABLE produto;
-- DROP TABLE tempo;
-- DROP TABLE vendas;

CREATE TABLE cliente (
    ID_Cliente  INT,
    Estado_Civil    VARCHAR(50),
    Sexo    VARCHAR(20),
    Bairro  VARCHAR(20));

CREATE TABLE produto (
    ID_Produto INT,
    COD_Produto INT,
    NM_Produto VARCHAR,
    Secao VARCHAR,
    Grupo VARCHAR,
    Subgrupo VARCHAR);

CREATE TABLE tempo (
    ID_Tempo INT,
    Ano INT,
    Mes INT,
    Dia INT,
    Data_sql DATE);

CREATE TABLE vendas (
    ID_Loja INT,
    ID_Produto INT,
    ID_Cliente INT,
    ID_Tempo INT,
    Qtde_Vendida INT,
    Receita_venda DECIMAL);

CREATE TABLE loja (
        ID_Loja int,
        COD_Loja int,
        NM_Loja varchar);
""")

CPU times: user 1.61 ms, sys: 0 ns, total: 1.61 ms
Wall time: 2.15 ms


In [5]:
%%time
duckdb.sql("""
INSERT INTO cliente SELECT * FROM cliente_df;
INSERT INTO vendas SELECT * FROM vendas_df;
INSERT INTO loja SELECT * FROM loja_df;
INSERT INTO produto SELECT * FROM produto_df;
INSERT INTO tempo SELECT * FROM tempo_df;
""")

duckdb.sql("""
--DROP INDEX IDx_Loja;
--DROP INDEX IDx_Produto;
--DROP INDEX IDx_Tempo;

--CREATE UNIQUE INDEX IDx_Loja ON loja (ID_Loja);
--CREATE UNIQUE INDEX IDx_Produto ON produto (ID_Produto);
--CREATE UNIQUE INDEX IDx_Tempo ON tempo (ID_Tempo);
--CREATE UNIQUE INDEX IDx_Cliente ON produto (ID_Cliente);
""")

duckdb.sql("""
-- SHOW tables
DESCRIBE

-- SELECT * FROM cliente;
-- SELECT * FROM loja;
-- SELECT * FROM produto;
-- SELECT * FROM tempo;
-- SELECT * FROM vendas;
""")

# duckdb.sql("""
# SHOW *
# FROM IDx_Loja;
# """)

CPU times: user 50 ms, sys: 5.03 ms, total: 55 ms
Wall time: 47.7 ms


┌────────────┬──────────────────────────────┬──────────────────────────────────────────────────────────────┬───────────┐
│ table_name │         column_names         │                         column_types                         │ temporary │
│  varchar   │          varchar[]           │                          varchar[]                           │  boolean  │
├────────────┼──────────────────────────────┼──────────────────────────────────────────────────────────────┼───────────┤
│ cliente    │ [ID_Cliente, Estado_Civil,…  │ [INTEGER, VARCHAR, VARCHAR, VARCHAR]                         │ false     │
│ loja       │ [ID_Loja, COD_Loja, NM_Loja] │ [INTEGER, INTEGER, VARCHAR]                                  │ false     │
│ produto    │ [ID_Produto, COD_Produto, …  │ [INTEGER, INTEGER, VARCHAR, VARCHAR, VARCHAR, VARCHAR]       │ false     │
│ tempo      │ [ID_Tempo, Ano, Mes, Dia, …  │ [INTEGER, INTEGER, INTEGER, INTEGER, DATE]                   │ false     │
│ vendas     │ [ID_Loja, ID_Prod

# Solicitações

-   Quantidade vendida no período entre 01/04/2020 a 01/04/2021; 
-   Quantidade de clientes únicos do sexo feminino e estado civil divorciada; 
-   Buscar os produtos com código 55,120,142 e que tiveram receita maior que
R$120,00; 
-   Verificar a receita da loja com o código 32 no período igual ou superior a
01/04/2021.

2.1) Quantidade vendida no período entre 01/04/2020 a 01/04/2021;

In [6]:
duckdb.sql("""
--DROP VIEW vwVendas_Tempo;

CREATE VIEW vwVendas_Tempo AS
SELECT
     Vendas.ID_Tempo,
	   Vendas.ID_Cliente,
	   Vendas.ID_Produto,
	   Vendas.Receita_venda,
	   Vendas.Qtde_Vendida,
	   Tempo.Data_sql AS Data

FROM vendas AS Vendas
LEFT JOIN tempo AS Tempo ON Vendas.ID_Tempo  = Tempo.ID_Tempo
""")

duckdb.sql("""
SELECT
      SUM(Qtde_Vendida) AS Qtde_Vendida

FROM vwVendas_Tempo
WHERE (Data > '2020-04-01' AND Data < '2022-04-01')
""")

┌──────────────┐
│ Qtde_Vendida │
│    int128    │
├──────────────┤
│         8629 │
└──────────────┘

2.2) Quantidade de clientes únicos do sexo feminino e estado civil divorciada;

In [7]:
duckdb.sql("""
SELECT 
  COUNT(DISTINCT(ID_Cliente)) AS Clientes,
  Sexo,
  Estado_Civil

FROM cliente
WHERE (Sexo = 'Feminino' and Estado_Civil = 'Divorciado')
GROUP BY Sexo, Estado_Civil;
""")

┌──────────┬──────────┬──────────────┐
│ Clientes │   Sexo   │ Estado_Civil │
│  int64   │ varchar  │   varchar    │
├──────────┼──────────┼──────────────┤
│        1 │ Feminino │ Divorciado   │
└──────────┴──────────┴──────────────┘

2.3) Buscar os produtos com código 55,120,142 e que tiveram receita maior que
R$120,00;

In [8]:
duckdb.sql("""
--DROP VIEW vwVendas_Produto;

CREATE VIEW vwVendas_Produto AS
SELECT Vendas.ID_Tempo,
	   Vendas.ID_Cliente,
	   Vendas.ID_Loja,
	   Vendas.ID_Produto,
	   Vendas.Receita_venda,
	   Vendas.Qtde_Vendida,
	   Produto.COD_Produto,
	   Produto.NM_Produto,
	   Produto.Secao,
	   Produto.Grupo,
	   Produto.Subgrupo

FROM produto AS Produto
LEFT JOIN vendas AS Vendas ON Produto.ID_Produto  = Vendas.ID_Produto
""")

duckdb.sql("""
SELECT 
      COD_Produto,
	    SUM(Receita_venda) AS Receita

FROM vwVendas_Produto
WHERE (COD_Produto in (55,120,142) and Receita_venda > 120)
GROUP BY COD_Produto;
""")

┌─────────────┬───────────────┐
│ COD_Produto │    Receita    │
│    int32    │ decimal(38,3) │
├─────────────┼───────────────┤
│          55 │       448.260 │
│         120 │      1115.490 │
│         142 │      1488.530 │
└─────────────┴───────────────┘

2.4) Verificar a receita da loja com o código 32 no período igual ou superior a
01/04/2021.

In [9]:
duckdb.sql("""
--DROP VIEW vwVendas_Loja;

CREATE VIEW vwVendas_Loja AS
SELECT
	   Loja.ID_Loja,
	   Loja.COD_Loja,      
	   Loja.NM_Loja,
     Vendas.ID_Tempo,
	   Vendas.ID_Cliente,
	   Vendas.ID_Produto,
	   Vendas.Receita_venda,
	   Vendas.Qtde_Vendida,
	   Tempo.Data_sql

FROM loja AS Loja
LEFT JOIN vendas AS Vendas ON Loja.ID_Loja  = Vendas.ID_Loja
LEFT JOIN tempo AS Tempo ON Vendas.ID_Tempo  = Tempo.ID_Tempo
""")

duckdb.sql("""
SELECT 
      COD_Loja,
      Data_sql

FROM vwVendas_Loja
WHERE COD_Loja = 32
--WHERE (COD_Loja = 32 and Data_sql > 2021-04-01)
GROUP BY COD_Loja, Data_sql;
""")

┌──────────┬────────────┐
│ COD_Loja │  Data_sql  │
│  int32   │    date    │
├──────────┼────────────┤
│       32 │ 2022-01-08 │
│       32 │ 2021-01-12 │
│       32 │ 2020-04-01 │
│       32 │ 2018-03-01 │
└──────────┴────────────┘